In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd  
from datetime import date

BASE_URL = "https://www.domiporta.pl/mieszkanie/sprzedam?Localizations%5B0%5D.Name=Katowice"
DOMAIN = "https://www.domiporta.pl"
data = date.today()

def get_total_pages(url):
    """Pobiera liczbę stron z głównej strony Domiporta."""
    headers = {"User-Agent": "Mozilla/5.0"}  
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Błąd: {response.status_code}")
        return 0
    
    soup = BeautifulSoup(response.text, "html.parser")
    
    pagination = soup.find_all("a", href=True)
    max_page = 0
    
    for tag in pagination:
        try:
            if "PageNumber=" in tag['href']: 
                page_number = int(tag.text.strip())
                max_page = max(max_page, page_number)
        except (ValueError, KeyError):
            continue
    
    if max_page > 0:
        print(f"Ostatnia strona: {max_page}")
        return max_page
    else:
        print("Nie udało się znaleźć numeru ostatniej strony.")
        return 0


def get_page_links(url, domain):
    """Pobiera linki z danej strony Domiporta."""
    headers = {"User-Agent": "Mozilla/5.0"} 
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Błąd: {response.status_code}")
        return []
    
    soup = BeautifulSoup(response.text, "html.parser")
    
    offer_links = soup.find_all("a", href=True)
    filtered_links = []
    
    for link in offer_links:
        href = link['href']
        full_url = domain + href if href.startswith("/") else href
        if full_url.startswith("https://www.domiporta.pl/nieruchomosci/sprzedam-"):
            filtered_links.append(full_url)
    
    return list(set(filtered_links))


def save_links_to_excel(links, filename):
    df = pd.DataFrame(links, columns=["Link"])  
    df.to_excel(filename, index=False, engine='openpyxl')  


def scrape_domiporta():
    total_pages = get_total_pages(BASE_URL)
    if total_pages == 0:
        print("Nie udało się pobrać liczby stron.")
        return
    
    print(f"Liczba stron: {total_pages}")
    
    all_links = []
    
    for page in range(1, total_pages + 1):
        url = f"{BASE_URL}&PageNumber={page}"
        print(f"Pobieram linki z strony {page}...")
        links = get_page_links(url, DOMAIN)
        all_links.extend(links)
    
    all_links = list(set(all_links))
    
    save_links_to_excel(all_links, rf"domiporta\domiporta_links_{data}.xlsx")
    
    print(f"Znaleziono {len(all_links)} unikalnych linków, zapisano je w pliku Excel.")
if __name__ == "__main__":
    scrape_domiporta()


Ostatnia strona: 36
Liczba stron: 36
Pobieram linki z strony 1...
Pobieram linki z strony 2...
Pobieram linki z strony 3...
Pobieram linki z strony 4...
Pobieram linki z strony 5...
Pobieram linki z strony 6...
Pobieram linki z strony 7...
Pobieram linki z strony 8...
Pobieram linki z strony 9...
Pobieram linki z strony 10...
Pobieram linki z strony 11...
Pobieram linki z strony 12...
Pobieram linki z strony 13...
Pobieram linki z strony 14...
Pobieram linki z strony 15...
Pobieram linki z strony 16...
Pobieram linki z strony 17...
Pobieram linki z strony 18...
Pobieram linki z strony 19...
Pobieram linki z strony 20...
Pobieram linki z strony 21...
Pobieram linki z strony 22...
Pobieram linki z strony 23...
Pobieram linki z strony 24...
Pobieram linki z strony 25...
Pobieram linki z strony 26...
Pobieram linki z strony 27...
Pobieram linki z strony 28...
Pobieram linki z strony 29...
Pobieram linki z strony 30...
Pobieram linki z strony 31...
Pobieram linki z strony 32...
Pobieram lin

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
from datetime import date

urls = pd.read_excel(rf'domiporta\domiporta_links_{data}.xlsx')
urls_list = urls['Link'].tolist()
random_urls = random.sample(urls_list, 10) if len(urls_list) >= 50 else urls_list
def safe_find_text(element):
    return element.get_text(strip=True) if element else None

def save_page_html(url):
    """Pobiera stronę i zapisuje jej HTML do słownika z danymi."""
    headers = {"User-Agent": "Mozilla/5.0"} 
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Błąd: {response.status_code}")
        return None
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    features_section = soup.find('section', class_='detials__section features')
    feature_containers = features_section.find_all('div', class_='features__container') if features_section else []

    features_data = {}

    for container in feature_containers:
        if container.find('ul'):
            features_list = container.find_all('li')
            for feature in features_list:
                name = safe_find_text(feature.find('span', class_='features__item_name'))
                value = safe_find_text(feature.find('span', class_='features__item_value'))
                if name and value:
                    features_data[name] = value
        elif container.find('dl'):
            feature_details = container.find_all('dl')
            for detail in feature_details:
                name = safe_find_text(detail.find('dt', class_='features__item_name'))
                value = safe_find_text(detail.find('dd', class_='features__item_value'))
                if name and value:
                    features_data[name] = value
    
    contact_section = soup.find('div', class_='contact__data_container details-databox')
    advertiser_name = safe_find_text(contact_section.find('p', class_='details-databox__name')) if contact_section else None
    advertiser_address = safe_find_text(contact_section.find('span')) if contact_section else None
    
    phone_elements = contact_section.find_all('div', class_='agent__phone') if contact_section else []
    phone_numbers = [phone['data-tel'] for phone in phone_elements if phone.get('data-tel')]

    property_info = {
        'Cena': safe_find_text(soup.find('span', {'itemprop': 'price'})),
        'Cena za m²': safe_find_text(soup.find('span', class_='summary__subtitle--price')),
        'Lokalizacja': safe_find_text(soup.find('span', itemprop='address')),
        'Powierzchnia': safe_find_text(soup.find('li', string=lambda x: x and 'Powierzchnia całkowita' in x)),
        'Liczba pokoi': safe_find_text(soup.find('li', string=lambda x: x and 'Liczba pokoi' in x)),
        'Piętro': safe_find_text(soup.find('li', string=lambda x: x and 'Piętro' in x)),
        'Typ budynku': safe_find_text(soup.find('li', string=lambda x: x and 'Typ budynku' in x)),
        'Rok budowy': safe_find_text(soup.find('li', string=lambda x: x and 'Rok budowy' in x)),
        'Materiał': safe_find_text(soup.find('li', string=lambda x: x and 'Materiał' in x)),
        'Ogłoszeniodawca': advertiser_name,
        'Adres ogłoszeniodawcy': advertiser_address,
        'Numery telefonów': ', '.join(phone_numbers),
        'URL': url
    }
    
    property_info.update(features_data)

    return property_info




def scrape_multiple_urls(urls):
    all_data = []

    for url in urls:
        data = save_page_html(url)
        if data:
            all_data.append(data)
        else:
            print(f"Nie udało się przetworzyć URL: {url}")

    df = pd.DataFrame(all_data)
    return df


df = scrape_multiple_urls(urls_list)
data = date.today()

df.to_clipboard()
data = date.today()
df.to_excel(rf'domiporta\domiporta_data_{data}.xlsx', index=False)

In [3]:
import pandas as pd

In [4]:
df = pd.read_excel(rf'domiporta\domiporta_data_{data}.xlsx')
df = df[['Cena', 'Cena za m2', 'Lokalizacja', 'Powierzchnia całkowita', 'Liczba pokoi', 'Piętro', 'Typ budynku', 'Rok budowy', 'Materiał', 'Ogłoszeniodawca', 'URL', 'Kategoria', 'Liczba pięter w budynku', 'Forma własności',	'Informacje dodatkowe:', 'Czynsz administracyjny', 'Powierzchnia piwnicy', 'Dostępne od']]

In [5]:
import pandas as pd


df = df.dropna(subset=['Lokalizacja'])

dzielnice = [
    'Śródmieście', 'Koszutka', 'Bogucice', 'Os. Paderewskiego - Muchowiec', 
    'Załęże', 'Osiedle Wincentego Witosa', 'Osiedle Tysiąclecia', 'Dąb', 
    'Wełnowiec-Józefowiec', 'Ligota-Panewniki', 'Brynów-Osiedle Zgrzebnioka', 
    'Załęska Hałda-Brynów', 'Zawodzie', 'Dąbrówka Mała', 'Szopienice-Burowiec', 
    'Janów-Nikiszowiec', 'Giszowiec', 'Murcki', 'Piotrowice-Ochojec', 
    'Zarzecze', 'Kostuchna', 'Podlesie', 'Centrum', 'Józefowiec', 'Brynów', 'Ligota', 'Piotrowice', 'Nikiszowiec', 'Ochojec', 'Szopienice'
]

dzielnice_set = set(dzielnice)

def process_lokalizacja(lokalizacja):
    parts = [part.strip() for part in lokalizacja.split(',')]

    if len(parts) < 2 or parts[0].lower() != 'śląskie' or parts[1].lower() != 'katowice':
        return None  
    
    wojewodztwo = parts[0]
    miasto = parts[1]
    dzielnica = "brak informacji"
    ulica = "brak informacji"
    
    if len(parts) == 2:
        return wojewodztwo, miasto, dzielnica, ulica

    elif len(parts) == 3:
        dodatkowy = parts[2]
        if dodatkowy in dzielnice_set:
            dzielnica = dodatkowy
        else:
            ulica = dodatkowy
        return wojewodztwo, miasto, dzielnica, ulica
    
    elif len(parts) >= 4:
        potencjalna_dzielnica = parts[2]
        potencjalna_ulica = parts[3]
        
        if potencjalna_dzielnica in dzielnice_set:
            dzielnica = potencjalna_dzielnica
        else:
            dzielnica = "brak informacji"
        
        if potencjalna_ulica:
            ulica = potencjalna_ulica
        return wojewodztwo, miasto, dzielnica, ulica

    else:
        potencjalna_dzielnica = parts[2]
        potencjalna_ulica = parts[3]
        
        if potencjalna_dzielnica in dzielnice_set:
            dzielnica = potencjalna_dzielnica
        else:
            dzielnica = "brak informacji"
        
        if potencjalna_ulica:
            ulica = potencjalna_ulica
        return wojewodztwo, miasto, dzielnica, ulica

df[['województwo', 'miasto', 'dzielnica', 'ulica']] = df['Lokalizacja'].apply(process_lokalizacja).apply(pd.Series)
df = df.dropna(subset=['województwo'])
df = df.reset_index(drop=True)

In [6]:
def clean_and_convert(column, to_remove):
    return pd.to_numeric(
        df[column]
        .str.replace(to_remove, '', regex=True)  
        .str.extract(r'(\d{1,3}(?:[\s,]?\d{3})*(?:[.,]\d+)?)')[0]  
        .str.replace(r'[^\d,]', '', regex=True)  
        .str.replace(r',', '.', regex=True) 
        .str.replace(r'\s+', '', regex=True)
    )


df['Cena'] = clean_and_convert('Cena', 'zł')
df['Cena za m2'] = clean_and_convert('Cena za m2', 'zł/m2')
df['Powierzchnia całkowita'] = clean_and_convert('Powierzchnia całkowita', 'm2')
df['Czynsz administracyjny'] = clean_and_convert('Czynsz administracyjny', 'zł')
df['Powierzchnia piwnicy'] = clean_and_convert('Powierzchnia piwnicy', 'm2')
df['Piętro'] = pd.to_numeric(df['Piętro'].str.replace('Parter', '0', regex=False))


In [7]:
df['Informacje dodatkowe'] = df['Informacje dodatkowe:'].str.split(', ')
features = set(sum([x for x in df['Informacje dodatkowe'] if isinstance(x, list)], []))
for feature in features:
    df[feature] = df['Informacje dodatkowe'].apply(lambda x: 1 if isinstance(x, list) and feature in x else 0)
df.drop(columns=['Informacje dodatkowe'], inplace=True)


In [8]:
prefix_groups = {
    'garaż_łącznie': 'garaż',  
    'ogrzewanie_łącznie': 'ogrzewanie',  
    'balkon_łącznie': ['taras', 'balkon'],  
    'miejsce_prakingowe_łącznie': ['miejsce parkingowe', 'parking_strzeżony']
}

for new_feature, group in prefix_groups.items():
    if isinstance(group, str):
        group_columns = [col for col in df.columns if col.lower().startswith(group.lower())]
    else:  
        group_columns = [col for col in group if col in df.columns]

    if group_columns:
        df[new_feature] = df[group_columns].max(axis=1)
        df.drop(columns=group_columns, inplace=True)  

threshold = 0.05 * len(df)
low_info_columns = [
    col for col in df.select_dtypes(include=['number']).columns
    if df[col].sum() < threshold
]
df.drop(columns=low_info_columns, inplace=True)

In [9]:
#df.to_excel(rf'C:\Users\PC\Desktop\scrapping_danych\domiporta\domiporta_data_preprocess.xlsx', index=False)
df.to_excel(rf'domiporta\data_archive\domiporta_data_preprocess_{data}.xlsx', index=False)